In [1]:
# import
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F

# sklearn
from sklearn.decomposition import PCA
from sklearn import preprocessing

# Pytorch
import torch
from torch.autograd import Variable
import torch.nn as nn
from torch.utils.data.sampler import BatchSampler, SubsetRandomSampler

# debuggin
from IPython.core.debugger import set_trace

print("import done!")

rng = np.random.random(23455)

import done!


In [2]:
# the model

class MultiFC(torch.nn.Module):
    
    def __init__(self, in_width, layers, out_width, activation, useNLL):
        assert(1 < len(layers))
        super(MultiFC, self).__init__()
        self.functions = []
        in_layer = in_width
        for out_layer in layers:
            self.functions.append(nn.Linear(in_layer, out_layer))
            in_layer = out_layer
        
        self.output_function = nn.Linear(in_layer, out_width)
        self.activation = activation
        self.useNLL = useNLL
    
    def forward(self, x):
        
        for function in self.functions:
            x = self.activation(function(x))

        if(self.useNLL):
            x = F.log_softmax(self.output_function(x))
        else:
            x = self.activation(self.output_function(x))
        return x
    
    def register_nan_checks(self):
        def check_grad(module, grad_input, grad_output):
            #  you can add this to see that the hook is called
            if any(np.all(np.isnan(gi.data.numpy())) for gi in grad_input if gi is not None):
                raise ValueError('NaN gradient in ' + type(module).__name__)
        self.apply(lambda module: module.register_backward_hook(check_grad))

In [3]:
# Model training
def train_model(training_x, training_target, valid_x, valid_target, layers, 
                classes, num_epochs, mini_batch_size, learning_rate, useNLL=True, activation=F.sigmoid):
    
    num_samples = training_x.shape[0]
    num_valid_samples = valid_x.shape[0]
    
    if(num_samples % mini_batch_size != 0):
        num_samples = int(num_samples/mini_batch_size) * mini_batch_size
    batches_per_epoch = int(num_samples/mini_batch_size)
    
    assert(num_samples > 0 and num_valid_samples > 0)
    
    print("Training a model:")
    print("learning_rate", learning_rate)
    print("layers", layers)
    print("num_epoch", num_epoch)
    print("mini_batch_size", mini_batch_size)    
    print("num samples", num_samples)
    print("num validation samples", num_valid_samples)
    
    # building the model
    assert(classes > 0)
    model = MultiFC(training_x.shape[1], layers, classes, activation, useNLL)

    # Convering torch to variables
    valid_x = Variable(valid_x)
    # training_target = Variable(training_target)
    # training_x = Variable(training_x)
    valid_target = Variable(valid_target)
    
    # loss function and update method
    if useNLL:
        criterion = nn.NLLLoss()
    else:
        criterion = nn.CrossEntropyLoss()
    
    assert(learning_rate > 0)
    optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)    

    
    # Statistics
    
    valid_accuracies = []
    valid_losses = []
    epoch_accuracies = [0.00]
    epoch_losses = [1]
    # epsilon
    eps = 0.001
    epoch = 0
    while eps < epoch_losses[-1] and epoch < num_epoch:
        
        # Batch sampler
        
        sampler = BatchSampler(SubsetRandomSampler(range(num_samples)), 
                               mini_batch_size, drop_last=True)
        running_loss = 0
        running_corrects = 0

        
        for indices in sampler:
            
            indices = torch.LongTensor(indices)

            target = Variable(training_target[indices], requires_grad=False)
            x = Variable(training_x[indices], requires_grad=False)

            optimizer.zero_grad()
            
            distribution = model(x)
            loss = criterion(distribution, target)           
            # Update parameters
            loss.backward()
            optimizer.step()
            
            # statistics
            _, preds = torch.max(distribution.data, 1)
            running_loss += loss.data[0]
            running_corrects += torch.sum(preds == target.data)
        
        epoch += 1
        
        # Validation
        distribution = model(valid_x)
        loss = criterion(distribution, valid_target)
        _, preds = torch.max(distribution.data, 1)
 
        # Statistics
        epoch_losses.append(running_loss / num_samples)
        epoch_accuracies.append(running_corrects / num_samples)
 
        valid_accuracies.append(torch.sum(preds == valid_target.data) / num_valid_samples)
        valid_losses.append(running_loss / num_valid_samples)

        if(epoch % 10 == 0 or epoch == num_epochs or epoch_losses[-1] <= eps):
            
            if(epoch == num_epochs or epoch_losses[-1] <= eps):
                print("==== FINAL VALIDATION ====")
            
            print("Epoch: {}, Epoch loss: {}, Epoch acc: {}, Valid loss: {}, Valid acc: {}".format(
                epoch, round(epoch_losses[-1], 6), round(epoch_accuracies[-1], 6), round(valid_losses[-1], 6),
                round(valid_accuracies[-1], 6)))
            
    epoch_accuracies = epoch_accuracies[1:]
    epoch_losses = epoch_losses[1:]
    
    return model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies

In [19]:
def generate_samples(valid_samples, classes=np.arange(1,11), pca=None, class_balanse=False,  num_features = 264):
    
    # Loading data
    data = np.array(pd.read_csv('./data/train_data.csv'))
    labels = np.array(pd.read_csv('./data/train_labels.csv'))

    # how many features
    data = data[:, 0:num_features]

    # remove classes
    indices = np.where((classes == labels))[0]
    data = data[indices]
    labels = labels[indices]
    
    # feature standardization
    data = preprocessing.scale(data)
    
    if not(pca is None):
        pca = PCA(n_components=pca)
        data = pca.fit_transform(data)
        print("PCA Results:")
        print("features", data.shape[1])
        print("pca.explained_variance_ratio_", pca.explained_variance_ratio_)
        print("sum(pca.explained_variance_ratio_)", sum(pca.explained_variance_ratio_))
    
    # sample indices
    indices = np.random.permutation(data.shape[0])
    
    valid_indices = indices[0: valid_samples]
    train_indices = indices[valid_samples:]
    
    # defining training data
    training_data = data[train_indices]
    training_labels = labels[train_indices]
    
    if(class_balanse):
        
        # lowest number of specific class samples
        samples_per_class = int(training_data.shape[0] / classes.shape[0])
        train_indices = np.zeros((classes.shape[0], samples_per_class))
        for index, c in enumerate(classes):
            class_sample_indices = np.where((c == training_labels))[0]
            train_indices[index] = np.random.choice(class_sample_indices, samples_per_class)

        train_indices = train_indices.ravel()
        train_indices = train_indices.astype(int)
        training_data = training_data[train_indices]
        training_labels = training_labels[train_indices]
    

    # Defining torch Tensor 
    training_x = torch.from_numpy(training_data).float()
    valid_x = torch.from_numpy(data[valid_indices]).float()
    
    # loss function requires values being 0 <= v <= c -1
    training_target = torch.from_numpy(training_labels[:,0] - 1)
    valid_target = torch.from_numpy(labels[valid_indices,0] -1)
    return training_x, training_target, valid_x, valid_target

In [20]:
class ModelHistory():
    def __init__(self):
        self.best_loss = 0
        self.best_accuracy = 0
        self.experiments = []

    def save_result_model(self, model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies, layers,
                    mini_batch_size, learning_rate, useNLL, activation, class_balanced):

        self.experiments.append({
            "model": model,
            "epoch_losses": epoch_losses,
            "epoch_accuracies": epoch_accuracies,
            "valid_losses": valid_losses,
            "valid_accuracies": valid_accuracies,
            "layers": layers, 
            "mini_batch_size": mini_batch_size,
            "learning_rate": learning_rate,
            "useNLL": useNLL,
            "activation": activation,
            "class_balanced": class_balanced
                           })

        if self.experiments[-1]["valid_losses"][-1] < self.experiments[self.best_loss]["valid_losses"][-1]:
            print("New BEST LOSS")
            self.best_loss = len(self.experiments) - 1  

        if self.experiments[-1]["valid_accuracies"][-1] > self.experiments[self.best_accuracy]["valid_accuracies"][-1]:
            print("New BEST ACCURACY")
            self.best_accuracy = len(self.experiments) - 1
        
ModelHistory = ModelHistory()

In [35]:
def test_model_save_result(model, pca=None):
    
    # Loading data
    data = np.genfromtxt('./data/test_data.csv', delimiter=',')
    num_samples = data.shape[0]
    print(num_samples)
    
    # feature standardization
    data = preprocessing.scale(data)
    
    if not(pca is None):
        pca = PCA(n_components=pca)
        data = pca.fit_transform(data)
        print("PCA Results:")
        print("features", data.shape[1])
        print("pca.explained_variance_ratio_", pca.explained_variance_ratio_)
        print("sum(pca.explained_variance_ratio_)", sum(pca.explained_variance_ratio_))
        
    # Defining torch Tensor 
    data = Variable(torch.from_numpy(data).float())
    
    # Result
    distribution = model(data)
    index, preds = torch.max(distribution.data, 1)
    predictions = preds + 1
    predictions.numpy()
    
    f = open('test_labels.csv', 'w+')
    f.write('Sample_id,Sample_label\n')

    ind = np.arange(num_samples)

    for i in range(predictions.shape[0]):
        row = "{},{}\n".format(ind[i]+1,int(predictions[i]))
        f.write(row)

    f.close()
    

### Experiment: Standardized Data

In [22]:
# Sample parameters (data size 4362)

valid_samples = 500
num_features = 264
class_balanse = False
classes = np.arange(1,11)  # type: list (classes 1 - 10) decreases training sample size (!!)
pca = None

# testing, validation and training samples
training_x, training_target, valid_x, valid_target = generate_samples(valid_samples, 
                                                                                           pca=pca, 
                                                                                   classes=classes,
                                                                                           class_balanse=class_balanse,
                                                                                           num_features=num_features)

In [39]:
# Hyperparameters

learning_rate = 0.15
num_epoch = 100
mini_batch_size = 100
activation = F.tanh
useNLL=True

layers = [600, 600]
model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies  = train_model(training_x, training_target, valid_x, valid_target, layers, classes.shape[0],
                num_epoch, mini_batch_size, learning_rate, useNLL=useNLL, activation=activation)

ModelHistory.save_result_model(model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies, layers,
                mini_batch_size, learning_rate, useNLL, activation, class_balanse)

Training a model:
learning_rate 0.15
layers [800, 400, 400]
num_epoch 1500
mini_batch_size 100
num samples 3800
num validation samples 500
Epoch: 10, Epoch loss: 0.012398, Epoch acc: 0.613158, Valid loss: 0.094226, Valid acc: 0.602
Epoch: 20, Epoch loss: 0.011572, Epoch acc: 0.633947, Valid loss: 0.087949, Valid acc: 0.626
Epoch: 30, Epoch loss: 0.011135, Epoch acc: 0.643684, Valid loss: 0.084624, Valid acc: 0.64
Epoch: 40, Epoch loss: 0.010841, Epoch acc: 0.652632, Valid loss: 0.082389, Valid acc: 0.642
Epoch: 50, Epoch loss: 0.010624, Epoch acc: 0.659211, Valid loss: 0.080744, Valid acc: 0.65
Epoch: 60, Epoch loss: 0.010452, Epoch acc: 0.668158, Valid loss: 0.079434, Valid acc: 0.654
Epoch: 70, Epoch loss: 0.010309, Epoch acc: 0.672895, Valid loss: 0.078351, Valid acc: 0.656
Epoch: 80, Epoch loss: 0.010186, Epoch acc: 0.674474, Valid loss: 0.077416, Valid acc: 0.664
Epoch: 90, Epoch loss: 0.010079, Epoch acc: 0.678684, Valid loss: 0.076602, Valid acc: 0.664
Epoch: 100, Epoch loss: 0.

Epoch: 880, Epoch loss: 0.008265, Epoch acc: 0.724737, Valid loss: 0.062817, Valid acc: 0.658
Epoch: 890, Epoch loss: 0.008256, Epoch acc: 0.726316, Valid loss: 0.062742, Valid acc: 0.658
Epoch: 900, Epoch loss: 0.008248, Epoch acc: 0.725789, Valid loss: 0.062688, Valid acc: 0.66
Epoch: 910, Epoch loss: 0.00824, Epoch acc: 0.726579, Valid loss: 0.062626, Valid acc: 0.66
Epoch: 920, Epoch loss: 0.008232, Epoch acc: 0.727895, Valid loss: 0.062566, Valid acc: 0.656
Epoch: 930, Epoch loss: 0.008226, Epoch acc: 0.727368, Valid loss: 0.062516, Valid acc: 0.654
Epoch: 940, Epoch loss: 0.008218, Epoch acc: 0.728158, Valid loss: 0.062459, Valid acc: 0.654
Epoch: 950, Epoch loss: 0.00821, Epoch acc: 0.728158, Valid loss: 0.0624, Valid acc: 0.66
Epoch: 960, Epoch loss: 0.008202, Epoch acc: 0.727368, Valid loss: 0.062337, Valid acc: 0.654
Epoch: 970, Epoch loss: 0.008195, Epoch acc: 0.730263, Valid loss: 0.062279, Valid acc: 0.654
Epoch: 980, Epoch loss: 0.008186, Epoch acc: 0.731053, Valid loss: 

In [36]:
# test result build with the model that achieved the best_loos validation (ModelHistory)

loss_index = ModelHistory.best_loss
model = ModelHistory.experiments[loss_index]["model"]

test_model_save_result(model, pca)

6544


In [ ]:
# Hyperparameters

learning_rate = 0.15
layers = [20, 20]
num_epoch = 100
mini_batch_size = 20
activation = F.tanh
useNLL=True

model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies  = train_model(training_x, training_target, valid_x, valid_target, layers, classes.shape[0],
                num_epoch, mini_batch_size, learning_rate, useNLL=useNLL, activation=activation)

ModelHistory.save_result_model(model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies, layers,
                mini_batch_size, learning_rate, useNLL, activation, class_balanse)

In [ ]:
# Hyperparameters

learning_rate = 0.15
layers = [50, 50]
num_epoch = 100
mini_batch_size = 20
activation = F.tanh
useNLL=True

model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies  = train_model(training_x, training_target, valid_x, valid_target, layers, classes.shape[0],
                num_epoch, mini_batch_size, learning_rate, useNLL=useNLL, activation=activation)

ModelHistory.save_result_model(model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies, layers,
                mini_batch_size, learning_rate, useNLL, activation, class_balanse)

In [ ]:
# Hyperparameters

learning_rate = 0.2
layers = [50, 50]
num_epoch = 100
mini_batch_size = 20
activation = F.tanh
useNLL=True

model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies  = train_model(training_x, training_target, valid_x, valid_target, layers, classes.shape[0],
                num_epoch, mini_batch_size, learning_rate, useNLL=useNLL, activation=activation)

ModelHistory.save_result_model(model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies, layers,
                mini_batch_size, learning_rate, useNLL, activation, class_balanse)

In [ ]:
# Hyperparameters

learning_rate = 0.15
layers = [80, 80]
num_epoch = 100
mini_batch_size = 20
activation = F.tanh
useNLL=True

model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies  = train_model(training_x, training_target, valid_x, valid_target, layers, classes.shape[0],
                num_epoch, mini_batch_size, learning_rate, useNLL=useNLL, activation=activation)

ModelHistory.save_result_model(model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies, layers,
                mini_batch_size, learning_rate, useNLL, activation, class_balanse)

In [ ]:
# Hyperparameters

learning_rate = 0.2
layer1_width = 80
layer2_width = 80
num_epoch = 100
mini_batch_size = 20
activation = F.tanh
useNLL=True

layers = [layer1_width, layer2_width]
model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies  = train_model(training_x, training_target, valid_x, valid_target, layers, classes.shape[0],
                num_epoch, mini_batch_size, learning_rate, useNLL=useNLL, activation=activation)

ModelHistory.save_result_model(model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies, layers,
                mini_batch_size, learning_rate, useNLL, activation, class_balanse)

In [ ]:
# Hyperparameters

learning_rate = 0.2
layer1_width = 100
layer2_width = 100
num_epoch = 100
mini_batch_size = 100
activation = F.tanh
useNLL=True


layers = [layer1_width, layer2_width]
model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies  = train_model(training_x, training_target, valid_x, valid_target, layers, classes.shape[0],
                num_epoch, mini_batch_size, learning_rate, useNLL=useNLL, activation=activation)

ModelHistory.save_result_model(model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies, layers,
                mini_batch_size, learning_rate, useNLL, activation, class_balanse)

In [ ]:
# Hyperparameters

learning_rate = 0.2
layer1_width = 80
layer2_width = 80
num_epoch = 200
mini_batch_size = 100
activation = F.tanh
useNLL=True

layers = [layer1_width, layer2_width]
model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies  = train_model(training_x, training_target, valid_x, valid_target, layers, classes.shape[0],
                num_epoch, mini_batch_size, learning_rate, useNLL=useNLL, activation=activation)

ModelHistory.save_result_model(model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies, layers,
                mini_batch_size, learning_rate, useNLL, activation, class_balanse)

In [ ]:
# Hyperparameters

learning_rate = 0.2
layer1_width = 80
layer2_width = 80
num_epoch = 100
mini_batch_size = 20
activation = F.tanh
useNLL=True

layers = [layer1_width, layer2_width]
model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies  = train_model(training_x, training_target, valid_x, valid_target, layers, classes.shape[0],
                num_epoch, mini_batch_size, learning_rate, useNLL=useNLL, activation=activation)

ModelHistory.save_result_model(model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies, layers,
                mini_batch_size, learning_rate, useNLL, activation, class_balanse)

In [ ]:
# Hyperparameters

learning_rate = 0.2
layer1_width = 80
layer2_width = 80
num_epoch = 100
mini_batch_size = 20
activation = F.tanh
useNLL=True

layers = [layer1_width, layer2_width]
model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies  = train_model(training_x, training_target, valid_x, valid_target, layers, classes.shape[0],
                num_epoch, mini_batch_size, learning_rate, useNLL=useNLL, activation=activation)

ModelHistory.save_result_model(model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies, layers,
                mini_batch_size, learning_rate, useNLL, activation, class_balanse)

In [ ]:
# Hyperparameters

learning_rate = 0.2
layer1_width = 120
layer2_width = 120
num_epoch = 300
mini_batch_size = 20
activation = F.tanh
useNLL=True

layers = [layer1_width, layer2_width]
model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies  = train_model(training_x, training_target, valid_x, valid_target, layers, classes.shape[0],
                num_epoch, mini_batch_size, learning_rate, useNLL=useNLL, activation=activation)

ModelHistory.save_result_model(model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies, layers,
                mini_batch_size, learning_rate, useNLL, activation, class_balanse)

In [ ]:
# Hyperparameters

learning_rate = 0.1
layer1_width = 80
layer2_width = 500
num_epoch = 100
mini_batch_size = 20
activation = F.tanh
useNLL=True

layers = [layer1_width, layer2_width]
model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies  = train_model(training_x, training_target, valid_x, valid_target, layers, classes.shape[0],
                num_epoch, mini_batch_size, learning_rate, useNLL=useNLL, activation=activation)

ModelHistory.save_result_model(model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies, layers,
                mini_batch_size, learning_rate, useNLL, activation, class_balanse)

In [ ]:
# Hyperparameters

learning_rate = 0.15
layer1_width = 1200
layer2_width = 1200
num_epoch = 200
mini_batch_size = 100
activation = F.tanh
useNLL=True

layers = [layer1_width, layer2_width]
model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies  = train_model(training_x, training_target, valid_x, valid_target, layers, classes.shape[0],
                num_epoch, mini_batch_size, learning_rate, useNLL=useNLL, activation=activation)

ModelHistory.save_result_model(model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies, layers,
                mini_batch_size, learning_rate, useNLL, activation, class_balanse)

In [ ]:
# Hyperparameters

learning_rate = 0.1
layer1_width = 300
layer2_width = 600
num_epoch = 100
mini_batch_size = 100
activation = F.tanh
useNLL=True

layers = [layer1_width, layer2_width]
model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies  = train_model(training_x, training_target, valid_x, valid_target, layers, classes.shape[0],
                num_epoch, mini_batch_size, learning_rate, useNLL=useNLL, activation=activation)

ModelHistory.save_result_model(model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies, layers,
                mini_batch_size, learning_rate, useNLL, activation, class_balanse)

In [ ]:
# Hyperparameters

learning_rate = 0.1
layer1_width = 600
layer2_width = 300
num_epoch = 100
mini_batch_size = 20
activation = F.tanh
useNLL=True


layers = [layer1_width, layer2_width]
model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies  = train_model(training_x, training_target, valid_x, valid_target, layers, classes.shape[0],
                num_epoch, mini_batch_size, learning_rate, useNLL=useNLL, activation=activation)

ModelHistory.save_result_model(model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies, layers,
                mini_batch_size, learning_rate, useNLL, activation, class_balanse)

In [ ]:
# Hyperparameters

learning_rate = 0.15
layer1_width = 800
layer2_width = 400
num_epoch = 400
mini_batch_size = 300
activation = F.tanh
useNLL=True

layers = [layer1_width, layer2_width]
model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies  = train_model(training_x, training_target, valid_x, valid_target, layers, classes.shape[0],
                num_epoch, mini_batch_size, learning_rate, useNLL=useNLL, activation=activation)

ModelHistory.save_result_model(model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies, layers,
                mini_batch_size, learning_rate, useNLL, activation, class_balanse)

In [ ]:
# Hyperparameters

learning_rate = 0.1
layer1_width = 800
layer2_width = 400
num_epoch = 300
mini_batch_size = 200
activation = F.tanh
useNLL=True

layers = [layer1_width, layer2_width]
model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies  = train_model(training_x, training_target, valid_x, valid_target, layers, classes.shape[0],
                num_epoch, mini_batch_size, learning_rate, useNLL=useNLL, activation=activation)

ModelHistory.save_result_model(model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies, layers,
                mini_batch_size, learning_rate, useNLL, activation, class_balanse)

In [ ]:
# Hyperparameters

learning_rate = 0.15
layer1_width = 400
layer2_width = 400
num_epoch = 200
mini_batch_size = 50
activation = F.tanh
useNLL=True

layers = [layer1_width, layer2_width]
model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies  = train_model(training_x, training_target, valid_x, valid_target, layers, classes.shape[0],
                num_epoch, mini_batch_size, learning_rate, useNLL=useNLL, activation=activation)

ModelHistory.save_result_model(model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies, layers,
                mini_batch_size, learning_rate, useNLL, activation, class_balanse)

In [ ]:
# Hyperparameters

learning_rate = 0.1
layer1_width = 500
layer2_width = 500
num_epoch = 100
mini_batch_size = 100
activation = F.tanh
useNLL=True

layers = [layer1_width, layer2_width]
model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies  = train_model(training_x, training_target, valid_x, valid_target, layers, classes.shape[0],
                num_epoch, mini_batch_size, learning_rate, useNLL=useNLL, activation=activation)

ModelHistory.save_result_model(model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies, layers,
                mini_batch_size, learning_rate, useNLL, activation, class_balanse)

In [ ]:
# Hyperparameters

learning_rate = 0.1
layer1_width = 400
layer2_width = 400
num_epoch = 100
mini_batch_size = 50
activation = F.tanh
useNLL=True

layers = [layer1_width, layer2_width]

model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies  = train_model(training_x, training_target, valid_x, valid_target, layers, classes.shape[0],
                num_epoch, mini_batch_size, learning_rate, useNLL=useNLL, activation=activation)

ModelHistory.save_result_model(model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies, layers,
                mini_batch_size, learning_rate, useNLL, activation, class_balanse)

In [ ]:
# Hyperparameters

learning_rate = 0.15
layers = [400, 200]
num_epoch = 100
mini_batch_size = 50
activation = F.tanh
useNLL=True

model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies  = train_model(training_x, training_target, valid_x, valid_target, layers, classes.shape[0],
                num_epoch, mini_batch_size, learning_rate, useNLL=useNLL, activation=activation)

save_result_model(model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies, layers,
                mini_batch_size, learning_rate, useNLL, activation, class_balanse)

In [ ]:
# Hyperparameters

learning_rate = 0.1
layers = [200, 200, 200]
num_epoch = 1000
mini_batch_size = 50
activation = F.tanh
useNLL=True

layers = [layer1_width, layer2_width]
model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies  = train_model(training_x, training_target, valid_x, valid_target, layers, classes.shape[0],
                num_epoch, mini_batch_size, learning_rate, useNLL=useNLL, activation=activation)

ModelHistory.save_result_model(model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies, layers,
                mini_batch_size, learning_rate, useNLL, activation, class_balanse)

### Experiment Standardized and class balanced data

In [ ]:
# Sample parameters (data size 4362)

valid_samples = 100
testing_samples = 100 # rest of data for training
num_features = 264
class_balanse = True
classes = np.arange(1,11)  # type: list (classes 1 - 10) decreases training sample size (!!)
pca = False

# testing, validation and training samples
training_x, training_target, valid_x, valid_target, test_x, test_target = generate_samples(valid_samples, 
                                                                                           testing_samples,
                                                                                           pca=pca, 
                                                                                           classes=classes,
                                                                                           class_balanse=class_balanse,
                                                                                           num_features=num_features)

In [ ]:
# Hyperparameters

learning_rate = 0.1
layers = [200, 200, 200]
num_epoch = 1000
mini_batch_size = 50
activation = F.tanh
useNLL=True

layers = [layer1_width, layer2_width]
model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies  = train_model(training_x, training_target, valid_x, valid_target, layers, classes.shape[0],
                num_epoch, mini_batch_size, learning_rate, useNLL=useNLL, activation=activation)

ModelHistory.save_result_model(model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies, layers,
                mini_batch_size, learning_rate, useNLL, activation, class_balanse)

In [ ]:
# Hyperparameters

learning_rate = 0.2
layer1_width = 80
layer2_width = 80
num_epoch = 100
mini_batch_size = 20
activation = F.tanh
useNLL=True

layers = [layer1_width, layer2_width]
model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies  = train_model(training_x, training_target, valid_x, valid_target, layers, classes.shape[0],
                num_epoch, mini_batch_size, learning_rate, useNLL=useNLL, activation=activation)

ModelHistory.save_result_model(model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies, layers,
                mini_batch_size, learning_rate, useNLL, activation, class_balanse)

In [ ]:
# Hyperparameters

learning_rate = 0.1
layer1_width = 80
layer2_width = 500
num_epoch = 100
mini_batch_size = 20
activation = F.tanh
useNLL=True

layers = [layer1_width, layer2_width]
model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies  = train_model(training_x, training_target, valid_x, valid_target, layers, classes.shape[0],
                num_epoch, mini_batch_size, learning_rate, useNLL=useNLL, activation=activation)

ModelHistory.save_result_model(model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies, layers,
                mini_batch_size, learning_rate, useNLL, activation, class_balanse)

In [ ]:
# Hyperparameters

learning_rate = 0.15
layer1_width = 600
layer2_width = 600
num_epoch = 100
mini_batch_size = 100
activation = F.tanh
useNLL=True

layers = [layer1_width, layer2_width]
model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies  = train_model(training_x, training_target, valid_x, valid_target, layers, classes.shape[0],
                num_epoch, mini_batch_size, learning_rate, useNLL=useNLL, activation=activation)

save_result_model(model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies, layers,
                mini_batch_size, learning_rate, useNLL, activation, class_balanse)

In [ ]:
# Hyperparameters

learning_rate = 0.15
layer1_width = 300
layer2_width = 300
num_epoch = 100
mini_batch_size = 100
activation = F.tanh
useNLL=True

layers = [layer1_width, layer2_width]
model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies  = train_model(training_x, training_target, valid_x, valid_target, layers, classes.shape[0],
                num_epoch, mini_batch_size, learning_rate, useNLL=useNLL, activation=activation)

ModelHistory.save_result_model(model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies, layers,
                mini_batch_size, learning_rate, useNLL, activation, class_balanse)

In [ ]:
# Hyperparameters

learning_rate = 0.15
layer1_width = 1200
layer2_width = 1200
num_epoch = 100
mini_batch_size = 100
activation = F.tanh
useNLL=True

layers = [layer1_width, layer2_width]
model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies  = train_model(training_x, training_target, valid_x, valid_target, layers, classes.shape[0],
                num_epoch, mini_batch_size, learning_rate, useNLL=useNLL, activation=activation)

ModelHistory.save_result_model(model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies, layers,
                mini_batch_size, learning_rate, useNLL, activation, class_balanse)

In [ ]:
# Hyperparameters

learning_rate = 0.15
layer1_width = 80
layer2_width = 80
num_epoch = 100
mini_batch_size = 100
activation = F.tanh
useNLL=True

layers = [layer1_width, layer2_width]
model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies  = train_model(training_x, training_target, valid_x, valid_target, layers, classes.shape[0],
                num_epoch, mini_batch_size, learning_rate, useNLL=useNLL, activation=activation)

ModelHistory.save_result_model(model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies, layers,
                mini_batch_size, learning_rate, useNLL, activation, class_balanse)

In [ ]:
# Hyperparameters

learning_rate = 0.15
layer1_width = 200
layer2_width = 200
num_epoch = 100
mini_batch_size = 100
activation = F.tanh
useNLL=True

layers = [layer1_width, layer2_width]
model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies  = train_model(training_x, training_target, valid_x, valid_target, layers, classes.shape[0],
                num_epoch, mini_batch_size, learning_rate, useNLL=useNLL, activation=activation)

ModelHistory.save_result_model(model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies, layers,
                mini_batch_size, learning_rate, useNLL, activation, class_balanse)

In [ ]:
# Hyperparameters

learning_rate = 0.15
layer1_width = 600
layer2_width = 600
num_epoch = 100
mini_batch_size = 100
activation = F.tanh
useNLL=True

layers = [layer1_width, layer2_width]
model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies  = train_model(training_x, training_target, valid_x, valid_target, layers, classes.shape[0],
                num_epoch, mini_batch_size, learning_rate, useNLL=useNLL, activation=activation)

ModelHistory.save_result_model(model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies, layers,
                mini_batch_size, learning_rate, useNLL, activation, class_balanse)

In [ ]:
# Hyperparameters

learning_rate = 0.15
layer1_width = 100
layer2_width = 600
num_epoch = 100
mini_batch_size = 100
activation = F.tanh
useNLL=True

layers = [layer1_width, layer2_width]
model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies  = train_model(training_x, training_target, valid_x, valid_target, layers, classes.shape[0],
                num_epoch, mini_batch_size, learning_rate, useNLL=useNLL, activation=activation)

ModelHistory.save_result_model(model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies, layers,
                mini_batch_size, learning_rate, useNLL, activation, class_balanse)

In [ ]:
# Hyperparameters

learning_rate = 0.15
layer1_width = 300
layer2_width = 600
num_epoch = 100
mini_batch_size = 100
activation = F.tanh
useNLL=True

layers = [layer1_width, layer2_width]
model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies  = train_model(training_x, training_target, valid_x, valid_target, layers, classes.shape[0],
                num_epoch, mini_batch_size, learning_rate, useNLL=useNLL, activation=activation)

ModelHistory.save_result_model(model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies, layers,
                mini_batch_size, learning_rate, useNLL, activation, class_balanse)

In [ ]:
# Hyperparameters

learning_rate = 0.15
layer1_width = 600
layer2_width = 300
num_epoch = 100
mini_batch_size = 100
activation = F.tanh
useNLL=True

layers = [layer1_width, layer2_width]
model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies  = train_model(training_x, training_target, valid_x, valid_target, layers, classes.shape[0],
                num_epoch, mini_batch_size, learning_rate, useNLL=useNLL, activation=activation)

ModelHistory.save_result_model(model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies, layers,
                mini_batch_size, learning_rate, useNLL, activation, class_balanse)

In [ ]:
# Hyperparameters

learning_rate = 0.15
layer1_width = 80
layer2_width = 100
num_epoch = 100
mini_batch_size = 100
activation = F.tanh
useNLL=True

layers = [layer1_width, layer2_width]
model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies  = train_model(training_x, training_target, valid_x, valid_target, layers, classes.shape[0],
                num_epoch, mini_batch_size, learning_rate, useNLL=useNLL, activation=activation)

ModelHistory.save_result_model(model, epoch_losses, epoch_accuracies, valid_losses, valid_accuracies, layers,
                mini_batch_size, learning_rate, useNLL, activation, class_balanse)